<a href="https://colab.research.google.com/github/Scott-Huston/DS-Unit-2-Applied-Modeling/blob/master/Copy_of_assignment_applied_modeling_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Applied Modeling, Module 3

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] Continue to iterate on your project: data cleaning, exploration, feature engineering, modeling.
- [ ] Make at least 1 partial dependence plot to explain your model.
- [ ] Share at least 1 visualization on Slack.

(If you have not yet completed an initial model yet for your portfolio project, then do today's assignment using your Tanzania Waterpumps model.)

## Stretch Goals
- [ ] Make multiple PDPs with 1 feature in isolation.
- [ ] Make multiple PDPs with 2 features in interaction.
- [ ] If you log-transformed your regression target, then convert your PDP back to original units. 
- [ ] Use Plotly to make a 3D PDP.
- [ ] Make PDPs with categorical feature(s). Use Ordinal Encoder, outside of a pipeline, to encode your data first. If there is a natural ordering, then take the time to encode it that way, instead of random integers. Then use the encoded data with pdpbox.I Get readable category names on your plot, instead of integer category codes.

## Links
- [Christoph Molnar: Interpretable Machine Learning — Partial Dependence Plots](https://christophm.github.io/interpretable-ml-book/pdp.html) + [animated explanation](https://twitter.com/ChristophMolnar/status/1066398522608635904)
- [Kaggle / Dan Becker: Machine Learning Explainability — Partial Dependence Plots](https://www.kaggle.com/dansbecker/partial-plots)
- [Plotly: 3D PDP example](https://plot.ly/scikit-learn/plot-partial-dependence/#partial-dependence-of-house-value-on-median-age-and-average-occupancy)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/Lambda School Data Science/NFL Play by Play 2009-2018 (v5).csv')
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,...,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackle_4_team,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,fumbled_1_team,fumbled_1_player_id,fumbled_1_player_name,fumbled_2_player_id,fumbled_2_player_name,fumbled_2_team,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,46,2009091000,PIT,TEN,PIT,home,TEN,TEN,30.0,2009-09-10,900.0,1800.0,3600.0,Half1,0,1,0,1,NaN,0.0,15:00,TEN 30,0,0,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,kickoff,0.0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PIT,39.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,893.0,1793.0,3593.0,Half1,0,1,0,1,1.0,0.0,14:53,PIT 42,10,5,(14:53) B.Roethlisberger pass short left to H....,pass,5.0,0,0,1.0,0,0,0,short,left,-3.0,8.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,2009-09-10,856.0,1756.0,3556.0,Half1,0,1,0,1,2.0,0.0,14:16,PIT 47,5,2,(14:16) W.Parker right end to PIT 44 for -3 ya...,run,-3.0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,right,end,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,815.0,1715.0,3515.0,Half1,0,1,0,1,3.0,0.0,13:35,PIT 44,8,2,(13:35) (Shotgun) B.Roethlisberger pass incomp...,pass,0.0,1,0,1.0,0,0,0,deep,right,34.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
4,139,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,807.0,1707.0,3507.0,Half1,0,1,0,1,4.0,0.0,13:27,PIT 44,8,2,(13:27) (Punt formation) D.Sepulveda punts 54 ...,punt,0.0,0,0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0


In [3]:
data.run_gap.unique()

array([nan, 'end', 'guard', 'tackle'], dtype=object)

In [4]:
data.pass_location.unique()

array([nan, 'left', 'right', 'middle'], dtype=object)

In [5]:
data.pass_length.unique()

array([nan, 'short', 'deep'], dtype=object)

In [6]:
data.play_type.unique()

array(['kickoff', 'pass', 'run', 'punt', 'field_goal', 'no_play', nan,
       'extra_point', 'qb_kneel', 'qb_spike'], dtype=object)

In [7]:
pd.options.display.max_rows = 300
data.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,449331,449332,449333,449334,449335,449336,449337,449338,449339,449340,449341,449342,449343,449344,449345,449346,449347,449348,449349,449350,449351,449352,449353,449354,449355,449356,449357,449358,449359,449360,449361,449362,449363,449364,449365,449366,449367,449368,449369,449370
play_id,46,68,92,113,139,162,183,207,228,253,277,301,322,346,365,393,425,449,470,496,522,542,563,584,603,622,646,681,705,726,750,774,800,821,842,868,889,905,929,950,...,3216,3239,3266,3295,3317,3339,3364,3386,3419,3444,3464,3483,3506,3528,3548,3573,3604,3637,3654,3679,3701,3723,3745,3770,3787,3809,3826,3848,3865,3887,3904,3949,3980,4002,4027,4059,4081,4106,4128,4150
game_id,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,2009091000,...,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700,2018121700
home_team,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,...,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR
away_team,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,...,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO
posteam,PIT,PIT,PIT,PIT,PIT,TEN,TEN,TEN,TEN,PIT,PIT,PIT,PIT,PIT,PIT,TEN,TEN,TEN,TEN,TEN,TEN,PIT,PIT,PIT,PIT,TEN,TEN,TEN,TEN,TEN,TEN,TEN,PIT,PIT,PIT,TEN,NaN,TEN,TEN,TEN,...,NO,NO,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NaN,NO,NO,NO,NO,NO,CAR,CAR,CAR,CAR,CAR,CAR,CAR,NO,NaN
posteam_type,home,home,home,home,home,away,away,away,away,home,home,home,home,home,home,away,away,away,away,away,away,home,home,home,home,away,away,away,away,away,away,away,home,home,home,away,NaN,away,away,away,...,away,away,home,home,home,home,home,home,home,home,home,away,away,away,away,away,away,away,away,away,away,away,away,away,away,NaN,away,away,away,away,away,home,home,home,home,home,home,home,away,NaN
defteam,TEN,TEN,TEN,TEN,TEN,PIT,PIT,PIT,PIT,TEN,TEN,TEN,TEN,TEN,TEN,PIT,PIT,PIT,PIT,PIT,PIT,TEN,TEN,TEN,TEN,PIT,PIT,PIT,PIT,PIT,PIT,PIT,TEN,TEN,TEN,PIT,NaN,PIT,PIT,PIT,...,CAR,CAR,NO,NO,NO,NO,NO,NO,NO,NO,NO,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,NaN,CAR,CAR,CAR,CAR,CAR,NO,NO,NO,NO,NO,NO,NO,CAR,NaN
side_of_field,TEN,PIT,PIT,PIT,PIT,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,TEN,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,TEN,TEN,TEN,TEN,TEN,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,PIT,...,CAR,CAR,NO,CAR,CAR,CAR,CAR,NO,CAR,NO,CAR,NO,NO,NO,NO,MID,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR,CAR
yardline_100,30,58,53,56,56,98,98,94,96,43,40,30,31,22,41,89,42,22,19,19,19,73,72,70,79,74,64,69,60,57,49,44,79,77,76,44,NaN,43,32,34,...,16,2,35,74,74,74,52,47,57,49,58,65,70,65,69,50,43,43,48,39,23,22,23,23,14,NaN,11,11,8,8,5,80,71,71,66,66,66,61,39,NaN
game_date,2009-09-10,2009-09-10,2009-09-10,2009-09-10,2009-09-10,2009-09-10,2009-09-10,20

In [0]:
future_columns = ['quarter_end', 'sp','ydsnet', 'desc', 'yards_gained', 
                'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble',
                'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 
                'run_location', 'field_goal_result', 'kick_distance', 'extra_point_result', 'run_gap', 'two_point_conv_result', 'timeout', 
               'timeout_team', 'td_team', 'posteam_score_post', 'defteam_score_post',
               'score_differential_post', 'epa', 'total_home_epa', 
                'total_away_epa', 'total_home_pass_epa', 'total_away_pass_epa', 
                'air_epa', 'yac_epa', 'comp_air_epa', 'comp_yac_epa', 
                'total_home_comp_air_epa', 'total_away_comp_air_epa', 
                'total_home_comp_yac_epa', 'total_home_raw_air_epa', 
                'total_away_raw_air_epa', 'total_home_raw_yac_epa', 
                'total_away_raw_yac_epa', 'wpa', 'home_wp_post', 'away_wp_post',
                'total_home_rush_wpa', 'total_away_rush_wpa', 
                'total_home_pass_wpa', 'total_away_pass_wpa', 'air_wpa', 
                'yac_wpa', 'comp_air_wpa', 'comp_yac_wpa', 
                'total_home_comp_air_wpa', 'total_away_comp_air_wpa', 
                'total_home_comp_yac_wpa', 'total_away_comp_yac_wpa', 
                'total_home_raw_air_wpa', 'total_away_raw_air_wpa', 
                'total_home_raw_yac_wpa', 'total_away_raw_yac_wpa', 
                'punt_blocked', 'first_down_rush', 'first_down_pass', 
                'third_down_converted', 'third_down_failed', 
                'fourth_down_converted', 'fourth_down_failed', 
                'incomplete_pass', 'interception', 'fumble_forced', 
                'fumble_not_forced', 'fumble_out_of_bounds', 'solo_tackle', 
                'safety', 'penalty', 'tackled_for_loss', 'fumble_lost', 
                'qb_hit', 'rush_attempt', 'pass_attempt', 'sack', 'touchdown', 
                'pass_touchdown', 'rush_touchdown', 'fumble', 'complete_pass', 
                'assist_tackle', 'lateral_reception', 'lateral_rush', 
                'lateral_return', 'lateral_recovery', 'passer_player_id', 
                'passer_player_name', 'receiver_player_id', 
                'receiver_player_name', 'rusher_player_id', 
                'rusher_player_name', 'lateral_receiver_player_id', 
                'lateral_receiver_player_name', 'lateral_rusher_player_name', 
                'lateral_sack_player_id', 'lateral_sack_player_name', 
                'interception_player_id', 'interception_player_name', 
                'lateral_interception_player_id', 
                'lateral_interception_player_name', 
                'tackle_for_loss_1_player_id', 'tackle_for_loss_1_player_name', 
                'tackle_for_loss_2_player_id', 'tackle_for_loss_2_player_name', 
                'qb_hit_1_player_id', 'qb_hit_1_player_name', 
                'qb_hit_2_player_id', 'qb_hit_2_player_name', 
                'forced_fumble_player_1_team', 'forced_fumble_player_1_player_id', 
                'forced_fumble_player_1_player_name', 'forced_fumble_player_2_team', 
                'forced_fumble_player_2_player_id', 'forced_fumble_player_2_player_name',
                'solo_tackle_1_player_id', 'solo_tackle_1_player_id', 
                'solo_tackle_1_player_name', 'solo_tackle_2_player_name', 
                'assist_tackle_1_player_id', 'assist_tackle_1_player_name',
                'assist_tackle_1_team', 'assist_tackle_2_player_id', 
                'assist_tackle_2_player_name', 'assist_tackle_2_team', 
                'assist_tackle_3_player_id', 'assist_tackle_3_player_name', 
                'assist_tackle_3_team', 'assist_tackle_4_player_id', 
                'assist_tackle_4_player_name', 'assist_tackle_4_team', 
                'pass_defense_1_player_id', 'pass_defense_1_player_name', 
                'pass_defense_2_player_id', 'pass_defense_2_player_name',
                'fumbled_1_team', 'fumbled_1_player_id', 'fumbled_1_player_name', 
                'fumbled_2_player_id', 'fumbled_2_player_name', 'fumbled_2_team', 
                'fumble_recovery_1_team', 'fumble_recovery_1_yards', 
                'fumble_recovery_1_player_id', 'fumble_recovery_1_player_name',
                'fumble_recovery_2_team', 'fumble_recovery_2_yards', 
                'fumble_recovery_2_player_id', 'fumble_recovery_2_player_name', 
                'penalty_team', 'penalty_player_id', 'penalty_player_name', 
                'penalty_yards', 'replay_or_challenge', 
                'replay_or_challenge_result', 'penalty_type', 
                'defensive_two_point_attempt', 'defensive_two_point_conv', 
                'defensive_extra_point_attempt', 'defensive_extra_point_conv', 
                'lateral_rusher_player_id', 'punt_returner_player_id', 
                'punt_returner_player_name', 'lateral_punt_returner_player_id', 
                'lateral_punt_returner_player_name', 'kickoff_returner_player_name', 
                'kickoff_returner_player_id', 'lateral_kickoff_returner_player_name',
                'punter_player_id', 'punter_player_name', 'kicker_player_name', 
                'kicker_player_id', 'own_kickoff_recovery_player_id', 
                'own_kickoff_recovery_player_name', 'blocked_player_id', 
                'blocked_player_name', 'solo_tackle_1_team', 'solo_tackle_2_team', 
                'solo_tackle_2_player_id', 'return_team', 'punt_inside_twenty', 
                'punt_in_endzone', 'punt_out_of_bounds', 'punt_downed', 
                'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone',
                'kickoff_out_of_bounds', 'kickoff_downed', 'kickoff_fair_catch', 
                'own_kickoff_recovery', 'own_kickoff_recovery_td', 'return_touchdown', 
                'extra_point_attempt', 'two_point_attempt', 'kickoff_attempt', 
                'punt_attempt', 'lateral_kickoff_returner_player_id', 'return_yards', 
                'first_down_penalty', 'total_home_rush_epa', 'total_away_rush_epa',
                'total_away_comp_yac_epa'
              ]     


target = 'play_type'

In [9]:
data[data.qb_scramble ==1]

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,...,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackle_4_team,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,fumbled_1_team,fumbled_1_player_id,fumbled_1_player_name,fumbled_2_player_id,fumbled_2_player_name,fumbled_2_team,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
108,2649,2009091000,PIT,TEN,PIT,home,TEN,TEN,43.0,2009-09-10,409.0,1309.0,1309.0,Half2,0,17,0,3,1.0,0.0,06:49,TEN 43,10,14,(6:49) B.Roethlisberger scrambles right end to...,run,3.0,0,0,1.0,0,0,1,NaN,NaN,NaN,NaN,right,end,NaN,...,00-0021230,D.Thornton,TEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
239,1556,2009091304,CLE,MIN,CLE,home,MIN,CLE,64.0,2009-09-13,200.0,200.0,2000.0,Half1,0,9,0,2,3.0,0.0,03:20,CLE 36,16,5,(3:20) (Shotgun) B.Quinn scrambles left end to...,run,11.0,1,0,1.0,0,0,1,NaN,NaN,NaN,NaN,left,end,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
334,3729,2009091304,CLE,MIN,CLE,home,MIN,CLE,80.0,2009-09-13,183.0,183.0,183.0,Half2,0,24,0,4,1.0,0.0,03:03,CLE 20,10,21,(3:03) (Shotgun) B.Quinn scrambles left guard ...,run,10.0,1,0,1.0,0,0,1,NaN,NaN,NaN,NaN,left,guard,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
539,193,2009091308,TB,DAL,DAL,away,TB,TB,35.0,2009-09-13,670.0,1570.0,3370.0,Half1,0,1,0,1,2.0,0.0,11:10,TB 35,19,31,(11:10) (Shotgun) T.Romo scrambles left end to...,run,2.0,1,0,1.0,0,0,1,NaN,NaN,NaN,NaN,left,end,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
601,1578,2009091308,TB,DAL,TB,home,DAL,TB,87.0,2009-09-13,80.0,80.0,1880.0,Half1,0,8,0,2,2.0,0.0,01:20,TB 13,11,0,(1:20) (Shotgun) B.Leftwich scrambles left end...,run,1.0,1,0,1.0,0,0,1,NaN,NaN,NaN,NaN,left,end,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
754,1225,2009091305,HOU,NYJ,HOU,home,NYJ,HOU,59.0,2009-09-13,626.0,626.0,2426.0,Half1,0,7,0,2,1.0,0.0,10:26,HOU 41,10,17,(10:26) M.Schaub scrambles up the middle to HO...,run,6.0,0,0,1.0,0,0,1,NaN,NaN,NaN,NaN,middle,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
835,3175,2009091305,HOU,NYJ,NYJ,away,HOU,NYJ,78.0,2009-09-13,702.0,702.0,702.0,Half2,0,18,0,4,2.0,0.0,11:42,NYJ 22,8,7,(11:42) M.Sanchez scrambles right end ran ob a...,run,5.0,0,0,1.0,0,0,1,NaN,NaN,NaN,NaN,right,end,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [0]:
# Changing the play type to pass if the qb scrambled as I'm trying to predict play calls, not play results

def scramble_to_pass(row):
  if row['qb_scramble']==1:
    row['play_type'] = 'pass'
  return row

data1=data.apply(scramble_to_pass, axis = 1)

In [11]:
# Checking that my scramble_to_pass function worked correctly
data1[data1.qb_scramble == 1]

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,...,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackle_4_team,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,fumbled_1_team,fumbled_1_player_id,fumbled_1_player_name,fumbled_2_player_id,fumbled_2_player_name,fumbled_2_team,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
108,2649,2009091000,PIT,TEN,PIT,home,TEN,TEN,43.0,2009-09-10,409.0,1309.0,1309.0,Half2,0,17,0,3,1.0,0.0,06:49,TEN 43,10,14,(6:49) B.Roethlisberger scrambles right end to...,pass,3.0,0,0,1.0,0,0,1,NaN,NaN,NaN,NaN,right,end,NaN,...,00-0021230,D.Thornton,TEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
239,1556,2009091304,CLE,MIN,CLE,home,MIN,CLE,64.0,2009-09-13,200.0,200.0,2000.0,Half1,0,9,0,2,3.0,0.0,03:20,CLE 36,16,5,(3:20) (Shotgun) B.Quinn scrambles left end to...,pass,11.0,1,0,1.0,0,0,1,NaN,NaN,NaN,NaN,left,end,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
334,3729,2009091304,CLE,MIN,CLE,home,MIN,CLE,80.0,2009-09-13,183.0,183.0,183.0,Half2,0,24,0,4,1.0,0.0,03:03,CLE 20,10,21,(3:03) (Shotgun) B.Quinn scrambles left guard ...,pass,10.0,1,0,1.0,0,0,1,NaN,NaN,NaN,NaN,left,guard,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
539,193,2009091308,TB,DAL,DAL,away,TB,TB,35.0,2009-09-13,670.0,1570.0,3370.0,Half1,0,1,0,1,2.0,0.0,11:10,TB 35,19,31,(11:10) (Shotgun) T.Romo scrambles left end to...,pass,2.0,1,0,1.0,0,0,1,NaN,NaN,NaN,NaN,left,end,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
601,1578,2009091308,TB,DAL,TB,home,DAL,TB,87.0,2009-09-13,80.0,80.0,1880.0,Half1,0,8,0,2,2.0,0.0,01:20,TB 13,11,0,(1:20) (Shotgun) B.Leftwich scrambles left end...,pass,1.0,1,0,1.0,0,0,1,NaN,NaN,NaN,NaN,left,end,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
754,1225,2009091305,HOU,NYJ,HOU,home,NYJ,HOU,59.0,2009-09-13,626.0,626.0,2426.0,Half1,0,7,0,2,1.0,0.0,10:26,HOU 41,10,17,(10:26) M.Schaub scrambles up the middle to HO...,pass,6.0,0,0,1.0,0,0,1,NaN,NaN,NaN,NaN,middle,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
835,3175,2009091305,HOU,NYJ,NYJ,away,HOU,NYJ,78.0,2009-09-13,702.0,702.0,702.0,Half2,0,18,0,4,2.0,0.0,11:42,NYJ 22,8,7,(11:42) M.Sanchez scrambles right end ran ob a...,pass,5.0,0,0,1.0,0,0,1,NaN,NaN,NaN,NaN,right,end,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [12]:
# Removing columns about the result of the play
data1 = data1.drop(future_columns, axis = 'columns')
data1.head().T


,0,1,2,3,4
play_id,46,68,92,113,139
game_id,2009091000,2009091000,2009091000,2009091000,2009091000
home_team,PIT,PIT,PIT,PIT,PIT
away_team,TEN,TEN,TEN,TEN,TEN
posteam,PIT,PIT,PIT,PIT,PIT
posteam_type,home,home,home,home,home
defteam,TEN,TEN,TEN,TEN,TEN
side_of_field,TEN,PIT,PIT,PIT,PIT
yardline_100,30,58,53,56,56
game_date,2009-09-10,2009-09-10,2009-09-10,2009-09-10,2009-09-10


In [13]:
# Filtering to only look at pass and run plays
df = data1[(data1['play_type']=='pass') | (data1['play_type']=='run')]
df.head()


,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,drive,qtr,down,goal_to_go,time,yrdln,ydstogo,play_type,shotgun,no_huddle,home_timeouts_remaining,away_timeouts_remaining,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,wp,def_wp,home_wp,away_wp,field_goal_attempt
1,68,2009091000,PIT,TEN,PIT,home,TEN,PIT,58.0,2009-09-10,893.0,1793.0,3593.0,Half1,1,1,1.0,0.0,14:53,PIT 42,10,pass,0,0,3,3,3.0,3.0,0,0,0.0,0.0,0.0,0.000969,0.108505,0.001061,0.169117,0.293700,0.003638,0.423011,0.0,0.0,2.338000,0.546433,0.453567,0.546433,0.453567,0.0
2,92,2009091000,PIT,TEN,PIT,home,TEN,PIT,53.0,2009-09-10,856.0,1756.0,3556.0,Half1,1,1,2.0,0.0,14:16,PIT 47,5,run,0,0,3,3,3.0,3.0,0,0,0.0,0.0,0.0,0.001057,0.105106,0.000981,0.162747,0.304805,0.003826,0.421478,0.0,0.0,2.415907,0.551088,0.448912,0.551088,0.448912,0.0
3,113,2009091000,PIT,TEN,PIT,home,TEN,PIT,56.0,2009-09-10,815.0,1715.0,3515.0,Half1,1,1,3.0,0.0,13:35,PIT 44,8,pass,1,0,3,3,3.0,3.0,0,0,0.0,0.0,0.0,0.001434,0.149088,0.001944,0.234801,0.289336,0.004776,0.318621,0.0,0.0,1.013147,0.510793,0.489207,0.510793,0.489207,0.0
5,162,2009091000,PIT,TEN,TEN,away,PIT,TEN,98.0,2009-09-10,796.0,1696.0,3496.0,Half1,2,1,1.0,0.0,13:16,TEN 2,10,run,0,0,3,3,3.0,3.0,0,0,0.0,0.0,0.0,0.002944,0.236918,0.025923,0.370074,0.144685,0.003099,0.216357,0.0,0.0,-1.398360,0.441071,0.558929,0.558929,0.441071,0.0
6,183,2009091000,PIT,TEN,TEN,away,PIT,TEN,98.0,2009-09-10,760.0,1660.0,3460.0,Half1,2,1,2.0,0.0,12:40,TEN 2,10,pass,0,0,3,3,3.0,3.0,0,0,0.0,0.0,0.0,0.003283,0.264022,0.024132,0.409397,0.116551,0.003305,0.179311,0.0,0.0,-2.094662,0.421547,0.578453,0.578453,0.421547,0.0


In [14]:
df.shape

(319721, 48)

In [15]:
df.isnull().sum()

play_id                         0
game_id                         0
home_team                       0
away_team                       0
posteam                        26
posteam_type                   26
defteam                        26
side_of_field                   0
yardline_100                   26
game_date                       0
quarter_seconds_remaining       0
half_seconds_remaining         17
game_seconds_remaining         14
game_half                       0
drive                           0
qtr                             0
down                          678
goal_to_go                     26
time                            0
yrdln                           0
ydstogo                         0
play_type                       0
shotgun                         0
no_huddle                       0
home_timeouts_remaining         0
away_timeouts_remaining         0
posteam_timeouts_remaining     26
defteam_timeouts_remaining     26
total_home_score                0
total_away_sco

In [16]:
# Getting majority baseline
df.play_type.value_counts()

print(186677/(186677+132692))

0.5845182218687475


In [0]:
# Splitting data into training, validation, and test sets
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.drop('play_type', axis = 'columns'), df['play_type'], train_size = .8, test_size = .2, stratify = df['play_type'])
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, train_size = .75, test_size = .25, stratify = y_train)

In [25]:
train = pd.concat([X_train, y_train], axis = 1)
train.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,drive,qtr,down,goal_to_go,time,yrdln,ydstogo,shotgun,no_huddle,home_timeouts_remaining,away_timeouts_remaining,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,wp,def_wp,home_wp,away_wp,field_goal_attempt,play_type
174941,3141,2012122304,JAC,NE,NE,away,JAC,NE,55.0,2012-12-23,189.0,1089.0,1089.0,Half2,20,3,2.0,0.0,03:09,NE 45,6,0,1,2,3,3.0,2.0,13,16,16.0,13.0,3.0,0.015915,0.107756,0.001130,0.163258,0.297523,0.003692,0.410726,0.0,0.0,2.306703,0.665513,0.334487,0.334487,0.665513,0.0,run
255207,2415,2014112309,SEA,ARI,SEA,home,ARI,SEA,64.0,2014-11-23,350.0,1250.0,1250.0,Half2,17,3,1.0,0.0,05:50,SEA 36,10,0,0,3,3,3.0,3.0,12,3,12.0,3.0,9.0,0.009805,0.122216,0.001703,0.186774,0.273653,0.003548,0.402301,0.0,0.0,1.966688,0.841578,0.158422,0.841578,0.158422,0.0,run
428907,2579,2018102900,BUF,NE,BUF,home,NE,NE,39.0,2018-10-29,742.0,1642.0,1642.0,Half2,11,3,1.0,0.0,12:22:00,NE 39,10,1,0,3,3,3.0,3.0,3,9,3.0,9.0,-6.0,0.001056,0.062161,0.000192,0.096063,0.345594,0.003255,0.491678,0.0,0.0,3.625731,0.411179,0.588821,0.411179,0.588821,0.0,run
115619,1253,2011112007,GB,TB,TB,away,GB,TB,83.0,2011-11-20,626.0,626.0,2426.0,Half1,7,2,2.0,0.0,10:26,TB 17,13,0,0,2,2,2.0,2.0,14,7,7.0,14.0,-7.0,0.152975,0.182247,0.008317,0.274193,0.157311,0.003138,0.221819,0.0,0.0,-0.451789,0.239059,0.760941,0.760941,0.239059,0.0,run
42592,853,2010010307,MIN,NYG,MIN,home,NYG,NYG,45.0,2010-01-03,106.0,1006.0,2806.0,Half1,5,1,1.0,0.0,01:46,NYG 45,10,0,0,3,2,3.0,2.0,7,0,7.0,0.0,7.0,0.015745,0.070698,0.000311,0.106751,0.327076,0.003204,0.476214,0.0,0.0,3.361160,0.816882,0.183118,0.816882,0.183118,0.0,run


In [27]:
# Get Profile Report
import pandas_profiling
pandas_profiling.ProfileReport(train)

#check out support vector machines, naive-bayes-classifier

KeyboardInterrupt: ignored

In [18]:
# Making pipeline and fitting basic random forest
!pip install --upgrade category_encoders pandas-profiling plotly

from sklearn.pipeline import make_pipeline
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.0)


In [19]:
RF_pipeline = make_pipeline(ce.OrdinalEncoder()
              ,IterativeImputer()
              ,RandomForestClassifier())
RF_pipeline.fit(X_train, y_train)
print(RF_pipeline.score(X_val,y_val))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.7061224489795919


In [20]:
from sklearn.ensemble import GradientBoostingClassifier
GDB_pipeline = make_pipeline(ce.OrdinalEncoder()
                            ,IterativeImputer()
                            ,GradientBoostingClassifier())

GDB_pipeline.fit(X_train, y_train)
print(GDB_pipeline.score(X_val,y_val))

0.7397450934396748


In [21]:
X_train.nunique()

play_id                         5096
game_id                         2526
home_team                         35
away_team                         35
posteam                           35
posteam_type                       2
defteam                           35
side_of_field                     36
yardline_100                      99
game_date                        472
quarter_seconds_remaining        901
half_seconds_remaining          1801
game_seconds_remaining          3600
game_half                          3
drive                             38
qtr                                5
down                               4
goal_to_go                         2
time                            1802
yrdln                           1759
ydstogo                           45
shotgun                            2
no_huddle                          2
home_timeouts_remaining            8
away_timeouts_remaining            5
posteam_timeouts_remaining         4
defteam_timeouts_remaining         4
t

In [0]:
# from sklearn.linear_model import LogisticRegression

# Log_pipeline = make_pipeline(ce.OrdinalEncoder()
#                             ,IterativeImputer()
#                             ,LogisticRegression(max_iter = 5000, n_jobs=-1))

# Log_pipeline.fit(X_train, y_train)
# print(Log_pipeline.score(X_val,y_val))